In [ ]:
from __future__ import division, print_function

# Third-party
from astropy.constants import G
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.optimize import leastsq

# Custom
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

In [ ]:
linestyle = dict(marker='', color="#444444", linewidth=2, alpha=0.7)
datastyle = dict(marker='o', markersize=4, color='#3182bd', alpha=1., 
                 ecolor='#9ecae1',  capthick=0, linestyle='none', elinewidth=1.)

### Pull in mass measurements compiled from Oleg and Andreas:

In [ ]:
gnedin_tbl = np.genfromtxt("../data/gnedin_tbl.txt", delimiter=',', names=True, dtype=None)

In [ ]:
g_Menc = gnedin_tbl['Menc']
g_Menc_l = gnedin_tbl['neg_err']
g_Menc_u = gnedin_tbl['pos_err']

Andreas has circular velocity, so convert to mass

In [ ]:
kuepper_tbl = np.genfromtxt("../data/kuepper15_tbl.txt", delimiter=',', names=True, dtype=None)

In [ ]:
k_Menc = (kuepper_tbl['radius']*u.kpc * (kuepper_tbl['v_c']*u.km/u.s)**2 / G).to(u.Msun).value
k_Menc_l = k_Menc - (kuepper_tbl['radius']*u.kpc * ((kuepper_tbl['v_c']-kuepper_tbl['neg_err'])*u.km/u.s)**2 / G).to(u.Msun).value
k_Menc_u = (kuepper_tbl['radius']*u.kpc * ((kuepper_tbl['v_c']+kuepper_tbl['pos_err'])*u.km/u.s)**2 / G).to(u.Msun).value - k_Menc

In [ ]:
r = np.concatenate((gnedin_tbl['radius'], kuepper_tbl['radius']))
Menc = np.concatenate((g_Menc, k_Menc))
Menc_l = np.concatenate((g_Menc_l, k_Menc_l))
Menc_u = np.concatenate((g_Menc_u, k_Menc_u))

idx = Menc.argsort()
obs_Menc = Menc[idx]
obs_Menc_l = Menc_l[idx]
obs_Menc_u = Menc_u[idx]
obs_r = r[idx]

In [ ]:
# np.savetxt("/Users/adrian/projects/pal5-bar/data/MW-Menc.txt", 
#            np.vstack((obs_r, obs_Menc, obs_Menc_l, obs_Menc_u)).T, delimiter=',', 
#            header="r, Menc, Menc_neg_err, Menc_pos_err")

### First my by-eye fit of a potential model to these data:

In [ ]:
# Background Milky Way potential
mw_potential = gp.CCompositePotential()
M_nucl = 2E9
rs_nucl = 0.1
mw_potential['nucl'] =  gp.HernquistPotential(m=M_nucl, c=rs_nucl, units=galactic)
mw_potential['bulge'] = gp.HernquistPotential(m=5E9, c=1., units=galactic)
mw_potential['disk'] = gp.MiyamotoNagaiPotential(m=6.8E10*u.Msun, a=3*u.kpc, b=280*u.pc, 
                                                 units=galactic)

# for DM halo potential
M_h = 6E11 * u.Msun
rs_h = 16. * u.kpc
v_c = np.sqrt(((np.log(2.) - 0.5) * (G * M_h / rs_h)).decompose(galactic).value)
mw_potential['halo'] = gp.SphericalNFWPotential(v_c=v_c, r_s=rs_h, units=galactic)

In [ ]:
r = np.logspace(-3.5, 2.6, 1024)
xyz = np.zeros((3,r.size))
xyz[0] = r
menc = mw_potential.mass_enclosed(xyz*u.kpc)

plt.errorbar(obs_r, obs_Menc, yerr=[obs_Menc_l,obs_Menc_u], **datastyle)

plt.loglog(r, menc.value, **linestyle)
plt.xlim(10**-3.5, 10**2.6)
plt.ylim(10**6.5, 10**12.25)

plt.xlabel('$r$ [kpc]')
plt.ylabel('$M(<r)$ [M$_\odot$]')

In [ ]:
mw_potential = gp.CCompositePotential()
mw_potential['bulge'] = gp.HernquistPotential(m=5E9, c=1., units=galactic)
mw_potential['disk'] = gp.MiyamotoNagaiPotential(m=6.8E10*u.Msun, a=3*u.kpc, b=280*u.pc, 
                                                 units=galactic)
    
def get_mw_potential(log_M_halo, r_s, log_M_nucl, a):
    # Background Milky Way potential
    mw_potential['nucl'] =  gp.HernquistPotential(m=np.exp(log_M_nucl), c=a, units=galactic)
    v_c = np.sqrt(((np.log(2.) - 0.5) * (G * np.exp(log_M_halo)*u.Msun / (r_s*u.kpc))).decompose(galactic).value)
    mw_potential['halo'] = gp.SphericalNFWPotential(v_c=v_c, r_s=r_s, units=galactic)
    return mw_potential

In [ ]:
fit_robs = obs_r[2:]
fit_Mobs = obs_Menc[2:]
log_Mobs = np.log(fit_Mobs)
x0 = (np.log(M_h.to(u.Msun).value), rs_h.to(u.kpc).value, np.log(M_nucl), rs_nucl)

xyz = np.zeros((3,fit_robs.size))

def f(p):
    pot = get_mw_potential(*p)
    
    xyz[0] = fit_robs
    log_menc = np.log(mw_potential.mass_enclosed(xyz).to(u.Msun).value)
    return (log_menc - log_Mobs) / 0.7

p_opt, ier = leastsq(f, x0=x0)
assert ier in range(1,4+1)

In [ ]:
p_opt

In [ ]:
fit_potential = get_mw_potential(*p_opt)

In [ ]:
r = np.logspace(-3.5, 2.6, 1024)
xyz = np.zeros((3,r.size))
xyz[0] = r
menc = fit_potential.mass_enclosed(xyz*u.kpc)

# observational points
fig,ax = plt.subplots(1,1,figsize=(6,6))

plt.errorbar(obs_r, obs_Menc, yerr=[obs_Menc_l,obs_Menc_u], **datastyle)
ax.loglog(r, menc.value, **linestyle)

ax.set_xlim(5E-3, 10**2.6)
ax.set_ylim(7E6, 10**12.25)

ax.set_xlabel('$r$ [kpc]')
ax.set_ylabel('$M(<r)$ [M$_\odot$]')

fig.tight_layout()

fig.savefig("../paper/figures/mass-profile.pdf")

In [ ]:
print("Halo mass: {:.2e} solMass, Scale radius: {:.2f} kpc".format(np.exp(p_opt[0]), p_opt[1]))

In [ ]:
print("Nuclear mass: {:.2e} solMass, Scale radius: {:.2f} pc".format(np.exp(p_opt[2]), 1000*p_opt[3]))

### Sanity check against circular velocity

In [ ]:
r = np.linspace(0.1, 250, 1024)
xyz = np.zeros((3,r.size))
xyz[0] = r
vcirc = fit_potential.circular_velocity(xyz*u.kpc)

plt.figure(figsize=(8,6))

plt.errorbar(kuepper_tbl['radius'], kuepper_tbl['v_c'], 
             yerr=(kuepper_tbl['neg_err'],kuepper_tbl['pos_err']),
             linestyle='none', marker='o', color='k', ecolor='#aaaaaa')

plt.plot(r, vcirc.to(u.km/u.s).value, marker='', 
         color=linecolor, linewidth=2., alpha=0.75)


plt.xlim(0, 225)
plt.ylim(45, 305)